<a href="https://colab.research.google.com/github/LeeSeungwon89/Kaggle_Dacon_Practice/blob/main/2.%20Porto_Seguro%E2%80%99s_Safe_Driver_Prediction_baseline_model1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install kaggle
from google.colab import files
files.upload()

In [ ]:
ls -1ha kaggle.json

kaggle.json


In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

# Permission Warning이 발생하지 않도록 해줍니다.
!chmod 600 ~/.kaggle/kaggle.json

# 참가한 대회 리스트를 확인합니다.
# !kaggle competitions list

In [ ]:
!kaggle competitions download -c porto-seguro-safe-driver-prediction

 98% 75.0M/76.5M [00:00<00:00, 153MB/s]
100% 76.5M/76.5M [00:00<00:00, 135MB/s]


In [ ]:
!ls

kaggle.json  porto-seguro-safe-driver-prediction.zip  sample_data


In [ ]:
!unzip porto-seguro-safe-driver-prediction.zip

Archive:  porto-seguro-safe-driver-prediction.zip
  inflating: sample_submission.csv   
  inflating: test.csv                
  inflating: train.csv               
